# Imports

In [211]:
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import collections
import networkx as nx

import pdb

pd.set_option('mode.chained_assignment', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

os.chdir("/home/jacobheglund/dev/raildelays")
print(os.getcwd())

/home/jacobheglund/dev/raildelays


In [212]:
# #TODO 2017 includes XXXPAD and PADXXX trips, remove these or get updated data from Teddy
# ## actually, my system of available routes doesn't scale well to include these, so just remove the data lol
# path = "./data/raw/HSP_2017_DID_PAD.txt"
# df_test = pd.read_csv(path)
# df_test["OD"] = df_test["station_origin"] + df_test["station_destination"]

# np.unique(df_test["OD"])




In [257]:
#TODO append the two data sources (2016, 2017) and use Teddy's maps csv to scale the stop position and 
# include all these data points

raw_data_path = "./data/raw/HSP_2016_DID_PAD.txt"
adjlist_path = "./data/processed/raildelays/raildelays_adjlist.txt"
adj_path = "./data/processed/raildelays/raildelays_adj.npy"
dataset_path = "./data/processed/raildelays/raildelays.npy"

# plotting options
show_plots = False
figsize = (16, 9)
dpi = 200
node_size = 50
# xlim, ylim based on min/max lat/long for the included stations in the dataset
xlim = (-4.5, 0.5)
ylim = (50.75, 52.25)

# Link-Based Node Formulation


## Data Preprocssing

In [258]:
df = pd.read_csv(raw_data_path, sep=",")

In [259]:
# this data was acquired using the station codes to find the station names using the link below,
# look them up in Google Maps, and get a lat/long coordinate of the station from Google Maps
## https://en.wikipedia.org/wiki/UK_railway_stations_%E2%80%93_A
#TODO replace this with ./data/teddy/stations.csv
# station map may be found at https://www.gwr.com/plan-journey/stations-and-routes 
stop_position ={\
"AML": ( 51.517207, -0.266751 ),
"APF": ( 51.639810, -1.242371 ),
"BAN": ( 52.060522, -1.327883 ),
"BGN": ( 51.507162, -3.575304 ),
"BNM": ( 51.522150, -0.645280 ),
"BPW": ( 51.514626, -2.542716 ),
"BRI": ( 51.448520, -2.582950 ),
"BTH": ( 51.377813, -2.356989 ),
"BWT": ( 51.127593, -2.990609 ),
"CDF": ( 51.475990, -3.178987 ),
"CMN": ( 51.853425, -4.305995 ),
"CHO": ( 51.569495, -1.156961 ),
"CPM": ( 51.462407, -2.115287 ),
"CUM": ( 51.653834, -1.236464 ),
"DID": ( 51.611047, -1.242809 ),
"EAL": ( 51.514956, -0.301902 ),
"EXD": ( 50.729359, -3.543525 ),
"FYS": ( 51.768553, -4.369169 ),
"GOR": ( 51.521505, -1.132981 ),
"HAY": ( 51.503028, -0.420389 ),
"HIG": ( 51.218433, -2.972049 ),
"HYD": ( 51.918563, -1.299300 ),
"IVR": ( 51.508453, -0.506122 ),
"KGS": ( 52.021025, -1.281072 ),
"KWL": ( 51.734407, -4.317376 ),
"KYN": ( 51.418035, -2.495436 ),
"LLE": ( 51.673900, -4.161347 ),
"LNY": ( 51.508032, -0.541948 ),
"MAI": ( 51.518727, -0.722769 ),
"NLS": ( 51.419460, -2.750466 ),
"NTH": ( 51.662220, -3.806959 ),
"NWP": ( 51.588709, -3.001131 ),
"OLF": ( 51.379242, -2.380790 ),
"OXF": ( 51.753483, -1.269902 ),
"PAD": ( 51.516683, -0.176897 ),
"PAN": ( 51.485475, -1.090419 ),
"PBY": ( 51.684164, -4.248703 ),
"PLY": ( 50.377667, -4.143794 ),
"PTA": ( 51.591775, -3.780789 ),
"RAD": ( 51.686207, -1.240432 ),
"RDG": ( 51.458404, -0.971437 ),
"SLO": ( 51.510596, -0.590391 ),
"STL": ( 51.505362, -0.378839 ),
"SWA": ( 51.625207, -3.941208 ),
"SWI": ( 51.565456, -1.785337 ),
"TAC": ( 51.881350, -1.297397 ),
"TAP": ( 51.523502, -0.681442 ),
"TAU": ( 51.023091, -3.102990 ),
"TLH": ( 51.470890, -1.029031 ),
"TOT": ( 50.435958, -3.688892 ),
"TVP": ( 50.917227, -3.359841 ),
"TWY": ( 51.475204, -0.862948 ),
"WEA": ( 51.513479, -0.319840 ),
"WDT": ( 51.510120, -0.472410 ),
"WNM": ( 51.348586, -2.942140 ),
"WOR": ( 51.358021, -2.909091 ),
"WSM": ( 51.344440, -2.972104 ),
"YAT": ( 51.390918, -2.827768 )}


stations = np.unique(df["station_curr"])
sps = dict(collections.Counter(df["station_curr"]))
sps = dict(sorted(sps.items()))

# the corridors are designed with this threshold in mind, it keeps out the "end of line" stations 
# which just add noise to the data
# in total this removes 1% of the data 

#TODO check if the set of filtered stops changes when 2017 is included,
## the corridors and available routes WILL change and that's not good
## I should update the corridors to fit the new data

# stations in 2016 data that are not in 2017 data
# AML
# PLY
# TOT

# stations in 2017 data that are not in 2016 data
# CBY
# DWL
# DWW
# EXT
# NTA
# PGN
# PWY
# SCS
# TGM
# TQY
# TRR

stop_thres = 50
for i in list(sps):
    if sps[i] < stop_thres:
        sps.pop(i)
        stop_position.pop(i)
        df_tmp = df.loc[df["station_curr"] == i]
        drop_list = list(df_tmp["RID"])
        for j in drop_list:
            df = df[df.RID != j]

df = df.reset_index()
df = df.drop(["index"], axis=1)

In [260]:
# - all corridors are defined in linear order
# -- once a train gets on a corridor, it travels through all the stations on that corridor
# --- although it does not necessarily stop at any of the stations on that corridor

# --------------------------------------------
# Northwest Corridor
# --------------------------------------------
# SWA - SWANSEA - START OF CORRIDOR
# NTH - NEATH
# PTA - PORT TALBOT PARKWAY
# BGN - BRIDGEND
# CDF - CARDIFF CENTRAL
# NWP - NEWPORT
# BPW - BRISTOL PARKWAY
# SWI - SWINDON - END OF CORRIDOR

# --------------------------------------------
# Southwest Corridor
# --------------------------------------------
# TAU - TAUNTON - START OF CORRIDOR
# BWT - BRIDGEWATER
# HIG - HIGHBRIDGE AND BURNHAM
# WSM - WESTON-SUPER-MARE
# WNM - WESTON MILTON
# WOR - WORLE
# YAT - YATTON
# NLS - NAILSEA AND BLACKWELL
# BRI - BRISTON TEMPLE MEADS
# BTH - BATH SPA
# CPM - CHIPPENHAM
# SWI - SWINDON - END OF CORRIDOR

# --------------------------------------------
# Middle Connecting Corridor
# --------------------------------------------
# SWI - SWINDON - START OF CORRIDOR
# DID - DIDCOT PARKWAY - END OF CORRIDOR

# --------------------------------------------
# North Central Corridor
# --------------------------------------------
# BAN - BANBURY - START OF CORRIDOR
# KGS - KINGS SUTTON
# HYD - HEYFORD
# TAC - TACKLEY
# OXF - OXFORD
# RAD - RADLEY
# CUM - CULHAM
# APF - APPLEFORD
# DID - DIDCOT PARKWAY - END OF CORRIDOR

# --------------------------------------------
# Eastern Corridor
# --------------------------------------------
# DID - DIDCOT PARKWAY
# CHO - CHOLSEY
# GOR - GORING AND STREATLEY
# PAN - PANGBOURNE
# TLH - TILEHURST
# RDG - READING
# TWY - TWYFORD
# MAI - MAIDENHEAD
# TAP - TAPLOW
# BNM - BURNHAM
# SLO - SLOUGH
# LNY - LANGLEY
# IVR - IVER
# WDT - WEST DRAYTON
# HAY - HAYES AND HARLINGTON
# STL - SOUTHALL
# EAL - EALING BROADWAY
# PAD - LONDON PADDINGTON

# used in defining the graph
corridor_list = [
    ["SWA", "NTH", "PTA", "BGN", "CDF", "NWP", "BPW", "SWI"],
    ["TAU", "BWT", "HIG", "WSM", "WNM", "WOR", "YAT", "NLS", "BRI", "BTH", "CPM", "SWI"],
    ["SWI", "DID"],
    ["BAN", "KGS", "HYD", "TAC", "OXF", "RAD", "CUM", "APF", "DID"],
    ["DID", "CHO", "GOR", "PAN", "TLH", "RDG", "TWY", "MAI", "TAP", "BNM", "SLO", "LNY",
     "IVR", "WDT", "HAY", "STL", "EAL", "PAD"]
]

# rearrange sps with stops grouped by corridor
sps_tmp = {}
for corridor in corridor_list:
    for j in corridor:
        keys = list(sps_tmp)
        if j not in keys:
            sps_tmp[j] = sps[j]
sps = sps_tmp

# used in finding links between stops
available_routes = [
    ["SWA", "NTH", "PTA", "BGN", "CDF", "NWP", "BPW", "SWI", "DID", "CHO",
     "GOR", "PAN", "TLH", "RDG", "TWY", "MAI", "TAP", "BNM", "SLO", "LNY",
     "IVR", "WDT", "HAY", "STL", "EAL", "PAD"],
    
    ["TAU", "BWT", "HIG", "WSM", "WNM", "WOR", "YAT", "NLS", "BRI", "BTH", 
     "CPM", "SWI", "DID", "CHO", "GOR", "PAN", "TLH", "RDG", "TWY", "MAI",
     "TAP", "BNM", "SLO", "LNY", "IVR", "WDT", "HAY", "STL", "EAL", "PAD"],
    
    ["BAN", "KGS", "HYD", "TAC", "OXF", "RAD", "CUM", "APF", "DID", "CHO",
     "GOR", "PAN", "TLH", "RDG", "TWY", "MAI", "TAP", "BNM", "SLO", "LNY",
     "IVR", "WDT", "HAY", "STL", "EAL", "PAD"]
]


In [261]:
def get_links(stop_1, stop_2, available_routes):
    # returns a list of all links between two stops on a route
    # this relies on having an array of possible routes
    #EX: my_list_1 = get_links("WDT", "PAD")
    # my_list_1 = ["WDTHHAY", "HAYSTL", "STLEAL", "EALPAD"]
    done = 0
    link_list = []
    for i in range(len(available_routes)):
        curr_route = available_routes[i]
        if (stop_1 in curr_route) and (stop_2 in curr_route) and (not done):
            done = 1
            stop_1_idx = curr_route.index(stop_1)
            stop_2_idx = curr_route.index(stop_2)            
            for j in range(stop_1_idx, stop_2_idx):                
                link_list.append(curr_route[j] + curr_route[j+1])
        
    return link_list


# my_list = get_links("NTH", "PAD", available_routes)
# my_list = get_links("DID", "PAD", available_routes)
# my_list = get_links("DID", "DID", available_routes)

# this situation shouldn't happen since all data ends at PAD
# my_list = get_links("PAD", "DID", available_routes)

# print(my_list)

In [262]:
# dataframe setup

# we're using scheduled times, but there is no reason that we couldn't use actual arrival/departure times
## the reason we don't is b/c we would have scheduled tiems available in the real world prediction
## maybe a better way would be to use scheduled times in the future predictions,
## and actual times in for input data but this just adds needless complecation for a first stab at the problem

# OD does not index the links, just the origin and destination of the route
df["OD"] = df["station_origin"] + df["station_destination"]

# get integer time values for all rows (arrival and departure)
df["dep_sched_int"] = df["departure_sched"]
df["dep_sched_int"][df["dep_sched_int"] == "terminating"] = df["arrival_sched"]
df["dep_actual_int"] = df["departure_actual"]
df["dep_actual_int"][df["dep_actual_int"] == "terminating"] = df["arrival_actual"]

# this one is different b/c "starting" arrival delay must be 0
df["arr_sched_int"] = df["arrival_sched"]
df["arr_sched_int"][df["arr_sched_int"] == "starting"] = df["departure_actual"]
df["arr_actual_int"] = df["arrival_actual"]
df["arr_actual_int"][df["arr_actual_int"] == "starting"] = df["departure_actual"]


# get scheduled arrival and departure as datetime
df["arr_sched_datetime"] = pd.to_datetime(df["date"].astype(str) + " " + df["arr_sched_int"].astype(str))
df["dep_sched_datetime"] = pd.to_datetime(df["date"].astype(str) + " " + df["dep_sched_int"].astype(str))
df["arr_actual_datetime"] = pd.to_datetime(df["date"].astype(str) + " " + df["arr_actual_int"].astype(str))
df["dep_actual_datetime"] = pd.to_datetime(df["date"].astype(str) + " " + df["dep_actual_int"].astype(str)) 

In [263]:
# find the delay that was generated by the the previous section of track for each stop
## arrival delay is positive if the train was late, and negative if the train was early
# get the set of links each train travelled over between consecutive stops

df["cum_arr_delay"] = ((df["arr_actual_datetime"] - df["arr_sched_datetime"]).dt.total_seconds()) / 60
init_list = []
for i in range(len(df)):
    init_list.append([])
df["prev_links"] = init_list

# make sure we're only combining data from one train trip at a time
for row_count in range(len(df)):
    curr_row = df[row_count:row_count+1]
    if (curr_row["arrival_sched"].item() == "starting"):
        df.at[row_count, "prev_link_datetime"] = df.at[row_count, "dep_sched_datetime"]
        df.at[row_count, "prev_link_delay"] = 0.
        df.at[row_count, "prev_links"] = []

    else:
        prev_row = df[row_count-1:row_count]

        t0 = prev_row["dep_sched_datetime"].item()
        t1 = curr_row["arr_sched_datetime"].item()

        # average time at which the train was running on previous link
        df.at[row_count, "prev_link_datetime"] = t0 + ((t1-t0) / 2)
        # df.at[row_count, "prev_link_datetime"] = pd.Timestamp(2002, 5, 5)
        df.at[row_count, "prev_link_delay"] = curr_row["cum_arr_delay"].item() - prev_row["cum_arr_delay"].item()
        df.at[row_count, "prev_links"] = get_links(prev_row["station_curr"].item(), curr_row["station_curr"].item(), available_routes)


    row_count += 1
        
# alphabetize columns
df = df.reindex(sorted(df.columns), axis=1)


## Generation and Visualization of Rail Network Graphs

In [264]:
counter = 0
stop_idx = {}
idx_stop = {}
for i in list(sps):
    stop_idx[i] = counter
    idx_stop[counter] = str(i)
    counter += 1

    

In [265]:
# # processing to verify the corridors I defined
# # Southwest Corridor
# # check if any trains starting in TAU go through BPW or if they all go through SWI
# ## Result: They all go through SWI, so the SW Corridor goes from TAU to SWI without branching off in this dataset
# ## Result: The Northwest and Southwest Corridors only meet at SWI for this dataset

# # North Central Corridor
# ## all trains going from BAN to PAD must pass through DID to be part of this dataset
# ## therefore there are no trains going from APF to CHO
# ## no trains go from BAN and pass through SWI

# df1 = df.copy()
# df1 = df1.loc[df1["station_origin"] == "BAN"]
# df1 = df1.loc[df1["station_curr"] == "SWI"]


# df1.head()
# print(len(df1))


In [266]:
# create graph representing rail network
directed_graph = False
#TODO add option for directed adjacency s.t. it follows the direction of traffic
## for this particular dataset where everything starts at XXX and ends at PAD
## i would need to see how other traffic papers accomplish this

total_str = ""
for corridor in corridor_list:
    if directed_graph:
        pass
    
    else:
        for j in range(len(corridor)):
            tmp_str = ""
            if (j == 0):
                tmp_str += str(stop_idx[corridor[j]])
                tmp_str += " "
                tmp_str += str(stop_idx[corridor[j+1]])

            elif (j == len(corridor)-1):
                tmp_str += str(stop_idx[corridor[j]])
                tmp_str += " "
                tmp_str += str(stop_idx[corridor[j-1]])

            else:
                tmp_str += str(stop_idx[corridor[j]])
                tmp_str += " "
                tmp_str += str(stop_idx[corridor[j-1]])
                tmp_str += " "
                tmp_str += str(stop_idx[corridor[j+1]])

            total_str += tmp_str
            total_str += "\n"
        
with open(adjlist_path, "w") as f:
    f.write(total_str)

In [267]:
idx_position = {}
counter = 0
for i in stop_position:
    idx_position[stop_idx[i]] = (stop_position[i][1],stop_position[i][0])
    

In [268]:
G = nx.read_adjlist(adjlist_path, nodetype=int)
if show_plots:
    fig = plt.subplots(figsize=figsize, dpi=dpi)
    nx.draw(G, idx_position, node_size=node_size)
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.show()

In [269]:
if show_plots:
    fig = plt.subplots(figsize=figsize, dpi=dpi)
    nx.draw_networkx_labels(G, idx_position, idx_stop, font_size=10)
    nx.draw_networkx_edges(G, idx_position)
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.show()


In [270]:
LG = nx.line_graph(G)

In [271]:
# get the position of the edges as average of station positions
edge_position = {}
for edge in LG.nodes():
    station_1 = int(edge[0])
    station_2 = int(edge[1])
    x1, x2 = idx_position[station_1][0], idx_position[station_2][0]
    y1, y2 = idx_position[station_1][1], idx_position[station_2][1]
    
    x_edge = (x1+x2)/2
    y_edge = (y1+y2)/2
    edge_position[edge] = (x_edge, y_edge) 


In [272]:
# get edge labels
edge_label = {}
for edge in LG.nodes():
    station_1 = int(edge[0])
    station_2 = int(edge[1])
    
    # TODO i'm not sure why these two cases mess it up, but
    # SWICPM should be CPMSWI
    # DIDAPF should be APFDID
    # something about the fact that they have multiple connections or are part of a complex junction

    if (idx_stop[station_1] == "SWI") and (idx_stop[station_2] == "CPM") or \
        (idx_stop[station_1] == "DID") and (idx_stop[station_2] == "APF"):
        station_tmp = station_1
        station_1 = station_2
        station_2 = station_tmp
    
    edge_label[edge] = idx_stop[station_1] + idx_stop[station_2]
    
    

In [273]:
if show_plots:
    fig = plt.subplots(figsize=figsize, dpi=dpi)
    nx.draw(LG, edge_position, node_size=node_size)
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.show()

In [274]:
if show_plots:
    fig = plt.subplots(figsize=figsize, dpi=dpi)
    nx.draw_networkx_labels(LG, edge_position, edge_label, font_size=7)
    nx.draw_networkx_edges(LG, edge_position)
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.show()


In [275]:
# add self loops to graph
for i in LG.nodes:
    LG.add_edge(i, i)

adj = nx.adjacency_matrix(LG).todense()
adj = np.array(adj).astype("d")
np.save(adj_path, adj)
if show_plots:
    plt.matshow(adj)
    plt.show()
# TODO maybe change indexing of stations so that it's not alphabetical, but done with stations "close" in space
# (or on the same corridors) as having consecutive indicies
## all this will do is make the adj look nicer, which is probably desirable for the paper just so people 
## don't get confused
## this didn't work, so whatever, I'll come back to it if it's important enough

## Extraction of Rail Link Features

In [276]:
# find starting and ending time
dt = 30 # minutes

def round_time(t_input, dt, direction="floor"):
    # 
    if direction == "floor":
        n_minutes = ((pd.to_timedelta(str(t_input[0:2] + ":" + t_input[2:4] + ":00")).seconds / 60) // dt * dt)
        return pd.to_timedelta(n_minutes, unit="m")
    
    elif direction == "ceil":
        n_minutes = ((pd.to_timedelta(str(t_input[0:2] + ":" + t_input[2:4] + ":00")).seconds / 60 + dt) // dt * dt)
        return pd.to_timedelta(n_minutes, unit="m")
    
t_min = min(df["dep_sched_int"])
t_max = max(df["dep_sched_int"])
t_start = round_time(t_min, dt)
t_end = round_time(t_max, dt, "ceil")

# unique times during each day
time_list = []
t_curr = t_start

while t_curr < t_end:
    time_list.append(t_curr)
    t_curr += pd.Timedelta(str(dt) + "minutes")

# unique dates of the year
date_list = np.unique(df["date"])

datetime_list = []
for i in date_list:
    t0 = pd.to_datetime(i + " 00:00:00")
        
    for j in time_list:
        t_curr = t0 + j
        datetime_list.append(t_curr)

In [277]:
## construct useful mappings

# mapping from node description to unique index
LG_node_idx = {}
# mapping from unique index to node description 
LG_idx_node = {}
counter = 0
for i in LG.nodes():
    LG_node_idx[i] = counter
    LG_idx_node[counter] = i
    counter += 1


# mapping from node description to label (i.e. "SWAPAD")
LG_node_label = edge_label
# mapping from label to node description
LG_label_node = {}
for i in LG_node_label:
    LG_label_node[LG_node_label[i]] = i


# mapping from time of year to time data index
datetime_idx = {}
# mapping from time data index to time of year
idx_datetime = {}
counter = 0
for i in datetime_list:
    datetime_idx[i] = counter
    idx_datetime[counter] = i
    counter += 1

In [278]:
n_nodes = len(adj)
n_timesteps = len(datetime_list)
n_features = 1 #  delay per link
n_timesteps_per_day = ((t_end - t_start)/dt).seconds / 60
print("n_timesteps_per_day:", n_timesteps_per_day)
delay_dataset = np.zeros((n_timesteps, n_nodes, n_features))
n_trains_total = np.zeros((n_nodes, 1))

for i in range(len(datetime_list)-1):
    # get set of trains running during current time period
    t0, t1 = datetime_list[i], datetime_list[i+1]
    df_tmp = df.loc[(t0 <= df["prev_link_datetime"]) & (df["prev_link_datetime"] <= t1)]
    n_trains_link = np.zeros((n_nodes, 1))

    for j in range(len(df_tmp)):
        curr_row = df_tmp[j:j+1]

        # get list of affected links
        prev_links = curr_row["prev_links"].item()
        n_trains = len(prev_links)

        for k in range(len(prev_links)):
            link = prev_links[k]
            link_idx = LG_node_idx[LG_label_node[link]]

            # sum the total delay experienced by trains along this link during the time period
            # those links into delay_dataset
            delay_dataset[i, link_idx, 0] += curr_row["prev_link_delay"].item()
            
            # enumerate trains that ran over each link during the time period
            n_trains_link[link_idx] += 1
            
    n_trains_total += n_trains_link
    # get the delay over each link averaged by number of trains that passed through the link
    divisor = n_trains_link + np.where(n_trains_link == 0, 1, 0)
    
    delay_dataset[i, :, :] = delay_dataset[i, :, :] / divisor

# # average number of trains per day per link
# print(n_trains_total / len(date_list))
np.save(dataset_path, delay_dataset)

# delay_dataset.shape = (n_timesteps_during_data_period, n_nodes, n_input_features)

n_timesteps_per_day: 15.0
